In [1]:
import pandas as pd
import numpy as np
import os

from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [2]:
source = 'source/water_use.csv'

In [3]:
data = pd.read_csv(source, skiprows=2, skip_footer=8, index_col=False)

/usr/local/lib/python3.5/site-packages/IPython/kernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [4]:
data.tail()

,Area,Area Id,Variable Name,Variable Id,Year,Value,Symbol,Md
9079,ZWE|Zimbabwe,181,Freshwater withdrawal as % of total renewable ...,4275,2002,21.02,NaN,NaN
9080,ZWE|Zimbabwe,181,Freshwater withdrawal as % of total renewable ...,4275,2007,17.85,NaN,NaN
9081,ZWE|Zimbabwe,181,Agricultural water withdrawal as % of total re...,4273,1987,4.82,NaN,NaN
9082,ZWE|Zimbabwe,181,Agricultural water withdrawal as % of total re...,4273,2002,16.59,NaN,NaN
9083,ZWE|Zimbabwe,181,Agricultural water withdrawal as % of total re...,4273,2007,14.65,NaN,NaN


In [5]:
data['Variable Name'].unique()

array(['Agricultural water withdrawal', 'Industrial water withdrawal',
       'Municipal water withdrawal', 'Total water withdrawal',
       'Irrigation water requirement',
       'Agricultural water withdrawal as % of total water withdrawal',
       'Industrial water withdrawal as % of total water withdrawal',
       'Municipal water withdrawal as % of total withdrawal',
       'Total water withdrawal per capita',
       'Fresh surface water withdrawal (primary and secondary)',
       'Fresh groundwater withdrawal (primary and secondary)',
       'Total freshwater withdrawal (primary and secondary)',
       'Freshwater withdrawal as % of total renewable water resources',
       'Agricultural water withdrawal as % of total renewable water resources',
       'Irrigation water withdrawal', 'Produced municipal wastewater',
       'Desalinated water produced', 'Collected municipal wastewater',
       'Treated municipal wastewater',
       'Number of municipal wastewater treatment facilitie

In [6]:
to_concept_id('Agricultural water withdrawal as % of total water withdrawal')

'agricultural_water_withdrawal_as_of_total_water_withdrawal'

In [3]:
source_dir = 'source'

In [4]:
all_data = []

for f in os.listdir(source_dir):
    if 'csv' in f:
        path = os.path.join(source_dir, f)
        all_data.append(pd.read_csv(path, skiprows=2, skip_footer=8, index_col=False, engine='python'))

In [5]:
all_concepts = []

for df in all_data:
    all_concepts.append(df[['Variable Name', 'Variable Id']].copy())

In [10]:
all_concepts = pd.concat(all_concepts)

In [11]:
all_concepts = all_concepts.reset_index(drop=True).drop_duplicates()

In [13]:
all_concepts['concept'] = all_concepts['Variable Name'].map(lambda x: to_concept_id(x))

all_concepts.columns = ['name', 'variable_id', 'concept']

In [14]:
all_concepts['concept'].map(len).max()

81

In [15]:
all_concepts['concept_type'] = 'measure'

In [16]:
all_concepts[['concept', 'concept_type', 'name', 'variable_id']].to_csv('../ddf--concepts--continuous.csv', index=False)

In [14]:
desc = pd.DataFrame([['name', 'Name', 'string'], 
                     ['year', 'Year', 'time'],
                     ['area', 'Area', 'entity_domain'],
                     ['area_id', 'Area Id', 'string']
                    ], columns=['concept', 'name', 'concept_type'])

In [15]:
desc.to_csv('../ddf--concepts--descrete.csv', index=False)

In [6]:
all_area = []

for df in all_data:
    all_area.append(df[['Area', 'Area Id']].copy())

In [7]:
all_area = pd.concat(all_area)

In [8]:
all_area = all_area.drop_duplicates()

In [10]:
all_area['area'] = all_area['Area'].map(lambda x: x.split('|')[0])
all_area['name'] = all_area['Area'].map(lambda x: x.split('|')[1])

In [11]:
all_area = all_area.drop('Area', axis=1)

In [13]:
all_area['area'] = all_area['area'].map(to_concept_id)

In [16]:
all_area.columns = ['area_id', 'area', 'name']

In [17]:
all_area[['area', 'name', 'area_id']].to_csv('../ddf--entities--area.csv', index=False)

In [21]:
for df in all_data:
    for g, ids in df.groupby('Variable Name').groups.items():
        df_concept = df.ix[ids].copy()
        concept = to_concept_id(g)
        
        df_concept['area'] = df['Area'].map(lambda x: to_concept_id(x.split('|')[0])) 
        df_concept = df_concept.rename(columns={'Value': concept, 'Year': 'year'})
        
        df_concept[['area', 'year', concept]].to_csv('../ddf--datapoints--{}--by--area--year.csv'.format(concept), 
                                                     index=False)

In [23]:
create_index_file('../')

,index,key,value,file
0,0,concept,concept_type,ddf--concepts--continuous.csv
1,1,concept,name,ddf--concepts--continuous.csv
2,2,concept,variable_id,ddf--concepts--continuous.csv
3,0,concept,name,ddf--concepts--descrete.csv
4,1,concept,concept_type,ddf--concepts--descrete.csv
5,0,"area,year",agricultural_water_withdrawal,ddf--datapoints--agricultural_water_withdrawal...
6,0,"area,year",agricultural_water_withdrawal_as_of_total_rene...,ddf--datapoints--agricultural_water_withdrawal...
7,0,"area,year",agricultural_water_withdrawal_as_of_total_wate...,ddf--datapoints--agricultural_water_withdrawal...
8,0,"area,year",agriculture_value_added_to_gdp,ddf--datapoints--agriculture_value_added_to_gd...
9,0,"area,year",arable_land,ddf--datapoints--arable_land--by--area--year.csv
